In [1]:
import json
import os
import pandas as pd    
import numpy as np 
import re
from numba import jit
from nltk.metrics import distance

In [2]:
with open("ang_en.txt", "r") as read_file:
    ang_en = read_file.readlines()

In [3]:
def pars(w):
    a = re.sub(r'[{}\n]', '', w).split('::')
    b = a[0].split(' ')
    return b[0], b[1], a[1]
f = [pars(w) for w in ang_en]
len(f)

15320

In [4]:
filtered = [ el for el in f if el[1] in ['n', 'v','vi','vt','vit', 'adj']]

In [5]:
filtered[:3]

[('ᛗᚫᚷᚫ', 'n', ' dative singular form of ᛗᚫᚷ'),
 ('Arabisc', 'adj', ' Arabic, Arabian'),
 ('Arabisc', 'adj', ' an Arab')]

In [6]:
filtered1 =[(re.sub(r'ƿ','w', el[0]), el[1], el[2].split(',')[0].split(';')[0]) for el in filtered]
len(filtered1)

13414

In [7]:
filtered2 = [ el for el in filtered1 if not re.match(r'.*alternative', el[2])]

In [8]:
filtered3 = [ el for el in filtered2 if not re.match(r'.*participle', el[2])]

In [9]:
filtered4 = [ el for el in filtered3 if not re.match(r'.*form', el[2])]
filtered4[-10:]

[('þyrelian', 'v', ' to make hollow'),
 ('þyrelung', 'n', ' perforation'),
 ('þyrre', 'adj', ' not wet'),
 ('þyrs', 'n', ' monster'),
 ('þyrstan',
  'v',
  ' to be thirsty (+genitive for something) (often impersonal'),
 ('þæc', 'n', ' roof'),
 ('þæc', 'n', ' thatch'),
 ('þæcele', 'n', ' torch'),
 ('þæctigele', 'n', ' roof tile'),
 ('þæsma', 'n', ' leaven')]

In [10]:
table = pd.DataFrame(filtered4)      
table

,0,1,2
0,Arabisc,adj,Arabic
1,Arabisc,adj,an Arab
2,Arabisc,n,the Arabic language
3,Bryttas,n,Britons
4,Crecisc,adj,Greek
...,...,...,...
10278,þæc,n,roof
10279,þæc,n,thatch
10280,þæcele,n,torch
10281,þæctigele,n,roof tile


In [11]:
table1 = table.drop_duplicates(subset = [0])
table1

,0,1,2
0,Arabisc,adj,Arabic
3,Bryttas,n,Britons
4,Crecisc,adj,Greek
6,Crist,n,Christ
7,Dene,n,Danes
...,...,...,...
10277,þyrstan,v,to be thirsty (+genitive for something) (ofte...
10278,þæc,n,roof
10280,þæcele,n,torch
10281,þæctigele,n,roof tile


In [12]:
table2 = table1[table1[0].str[0].str.islower()]
table2

,0,1,2
42,abecede,n,ABC
43,abelgan,v,to make angry
46,abeodan,v,to proclaim
48,aberan,v,to bear
50,aberendlic,adj,bearable
...,...,...,...
10277,þyrstan,v,to be thirsty (+genitive for something) (ofte...
10278,þæc,n,roof
10280,þæcele,n,torch
10281,þæctigele,n,roof tile


In [13]:
table2.to_csv('ang_en2.csv', index=False)

In [14]:
clean = table2.to_numpy()
clean

array([['abecede', 'n', ' ABC'],
       ['abelgan', 'v', ' to make angry'],
       ['abeodan', 'v', ' to proclaim'],
       ...,
       ['þæcele', 'n', ' torch'],
       ['þæctigele', 'n', ' roof tile'],
       ['þæsma', 'n', ' leaven']], dtype=object)

In [15]:
q = [el[0] for el in clean]
len(q)

6562

In [16]:
mono = [w for w in q if not re.match(r'([aoeiuyæ].*){3,}', w)]
len(mono)

5772

In [17]:
new_clean = pd.read_csv (r'ang_en_wiktionary - without adv.csv')
new_clean_1 = pd.DataFrame(new_clean, columns= ['word','pos', 'definition_edited'])
new_clean_1

,word,pos,definition_edited
0,abecede,n,ABC; alphabet
1,abelgan,v,"to make angry, tick off, enrage, irritate, ann..."
2,abeodan,v,"to proclaim, announce, state, bid"
3,aberan,v,"to bear, carry, tolerate, endure, suffer; thole"
4,aberendlic,adj,bearable; tolerable
...,...,...,...
6414,þyrstan,v,to be thirsty
6415,þæc,n,roof
6416,þæcele,n,"torch, light"
6417,þæctigele,n,roof tile


In [18]:
new_arr = new_clean_1.to_numpy()
new_arr

array([['abecede', 'n', 'ABC; alphabet'],
       ['abelgan', 'v',
        'to make angry, tick off, enrage, irritate, annoy, vex, grill, incense, cause anyone to swell with anger, offend'],
       ['abeodan', 'v', 'to proclaim, announce, state, bid'],
       ...,
       ['þæcele', 'n', 'torch, light'],
       ['þæctigele', 'n', 'roof tile'],
       ['þæsma', 'n', 'leaven; yeast']], dtype=object)

полный корпус

In [34]:
# полный корпус
new_arr_tot = [el[0] for el in new_arr]
len(new_arr_tot)

6419

выборка глаголов

In [20]:
verbs = [el[0] for el in new_arr if el[1]=='v']

In [21]:
with open("verbs.txt", 'w') as output:
    for row in verbs:
        output.write(str(row) + '\n')

In [22]:
new_arr_1 = [el[0] for el in new_arr if not el[1]=='v']
len(new_arr_1)

4593

In [23]:
with open("no_pref.txt", "r") as read_file:
    _verbs = read_file.readlines()

In [24]:
_verbs = [w.strip() for w in _verbs] 

корпус с глаголами без приставок

In [35]:
new = new_arr_1 + _verbs
len(new)

5440

корпус с односложными словами

In [36]:
# mono_new_2 = [w for w in new if not re.match(r'([aoeiuyæ].*?){3,}', w)]
# mono_new_2 = [(w, len(re.findall(r'[aoeiuyæ]+', w))) for w in new]
mono_new_2 = [w for w in new if len(re.findall(r'[aoeiuyæ]+', w)) < 3 ]
len(mono_new_2)

3790

In [27]:
with open("mono_2.txt", 'w') as output:
    for row in mono_new_2:
        output.write(str(row) + '\n')

левенштейн

In [28]:
from tqdm import tqdm

In [29]:
# nopython=True, 
@jit
def fisher_wagner(seq1, seq2):
    size_x = len(seq1) + 1
    size_y = len(seq2) + 1
    matrix = np.zeros((size_x, size_y))
    for x in range(size_x):
        matrix[x, 0] = x
    for y in range(size_y):
        matrix[0, y] = y

    for x in range(1, size_x):
        for y in range(1, size_y):
            if seq1[x-1] == seq2[y-1]:
                matrix[x, y] = min(
                    matrix[x-1, y] + 1,
                    matrix[x-1, y-1],
                    matrix[x, y-1] + 1)
            else:
                matrix[x, y] = min(
                    matrix[x-1, y] + 1,     # удаление
                    matrix[x-1, y-1] + 1,   # замена
                    matrix[x, y-1] + 1)     # вставка
    return matrix[size_x - 1, size_y - 1]

@jit(nopython=True, parallel=True)
def edit_distance(e1, e2):
    return distance.edit_distance(e1, e2)


In [30]:
# pairs_all = []
# for e1 in tqdm(new_arr_1):
#     for e2 in new_arr_1:
#         if e1 != e2:
#             pairs_all.append((e1, e2, fisher_wagner(e1, e2))) 
# #             pairs_all.append((e1, e2, edit_distance(e1, e2)))

In [31]:
# lev_table = pd.DataFrame(pairs)

In [32]:
# lev_table